In [ ]:
import streamlit as st
import tiktoken
from loguru import logger

from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import UnstructuredPowerPointLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

# from streamlit_chat import message
from langchain.callbacks import get_openai_callback
from langchain.memory import StreamlitChatMessageHistory

def main():
    st.set_page_config(
    page_title="DirChat",
    page_icon=":books:")

    st.title("_Private Data :red[QA Chat]_ :books:")

    if "conversation" not in st.session_state:
        st.session_state.conversation = None

    if "chat_history" not in st.session_state:
        st.session_state.chat_history = None

    if "processComplete" not in st.session_state:
        st.session_state.processComplete = None

    with st.sidebar:
        uploaded_files =  st.file_uploader("Upload your file",type=['pdf','docx'],accept_multiple_files=True)
        openai_api_key = st.text_input("OpenAI API Key", key="chatbot_api_key", type="password")
        process = st.button("Process")
    if process:
        if not openai_api_key:
            st.info("Please add your OpenAI API key to continue.")
            st.stop()
        files_text = get_text(uploaded_files)
        text_chunks = get_text_chunks(files_text)
        vetorestore = get_vectorstore(text_chunks)
     
        st.session_state.conversation = get_conversation_chain(vetorestore,openai_api_key) 

        st.session_state.processComplete = True

    if 'messages' not in st.session_state:
        st.session_state['messages'] = [{"role": "assistant", 
                                        "content": "안녕하세요! 주어진 문서에 대해 궁금하신 것이 있으면 언제든 물어봐주세요!"}]

    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    history = StreamlitChatMessageHistory(key="chat_messages")

    # Chat logic
    if query := st.chat_input("질문을 입력해주세요."):
        st.session_state.messages.append({"role": "user", "content": query})

        with st.chat_message("user"):
            st.markdown(query)

        with st.chat_message("assistant"):
            chain = st.session_state.conversation

            with st.spinner("Thinking..."):
                result = chain({"question": query})
                with get_openai_callback() as cb:
                    st.session_state.chat_history = result['chat_history']
                response = result['answer']
                source_documents = result['source_documents']

                st.markdown(response)
                with st.expander("참고 문서 확인"):
                    st.markdown(source_documents[0].metadata['source'], help = source_documents[0].page_content)
                    st.markdown(source_documents[1].metadata['source'], help = source_documents[1].page_content)
                    st.markdown(source_documents[2].metadata['source'], help = source_documents[2].page_content)
                    


# Add assistant message to chat history
        st.session_state.messages.append({"role": "assistant", "content": response})

def tiktoken_len(text):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    tokens = tokenizer.encode(text)
    return len(tokens)

def get_text(docs):

    doc_list = []
    
    for doc in docs:
        file_name = doc.name  # doc 객체의 이름을 파일 이름으로 사용
        with open(file_name, "wb") as file:  # 파일을 doc.name으로 저장
            file.write(doc.getvalue())
            logger.info(f"Uploaded {file_name}")
        if '.pdf' in doc.name:
            loader = PyPDFLoader(file_name)
            documents = loader.load_and_split()
        elif '.docx' in doc.name:
            loader = Docx2txtLoader(file_name)
            documents = loader.load_and_split()
        elif '.pptx' in doc.name:
            loader = UnstructuredPowerPointLoader(file_name)
            documents = loader.load_and_split()

        doc_list.extend(documents)
    return doc_list


def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=900,
        chunk_overlap=100,
        length_function=tiktoken_len
    )
    chunks = text_splitter.split_documents(text)
    return chunks


def get_vectorstore(text_chunks):
    embeddings = HuggingFaceEmbeddings(
                                        model_name="jhgan/ko-sroberta-multitask",
                                        model_kwargs={'device': 'cpu'},
                                        encode_kwargs={'normalize_embeddings': True}
                                        )  
    vectordb = FAISS.from_documents(text_chunks, embeddings)
    return vectordb

def get_conversation_chain(vetorestore,openai_api_key):
    llm = ChatOpenAI(openai_api_key=openai_api_key, model_name = 'gpt-3.5-turbo',temperature=0)
    conversation_chain = ConversationalRetrievalChain.from_llm(
            llm=llm, 
            chain_type="stuff", 
            retriever=vetorestore.as_retriever(search_type = 'mmr', vervose = True), 
            memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key='answer'),
            get_chat_history=lambda h: h,
            return_source_documents=True,
            verbose = True
        )

    return conversation_chain



if __name__ == '__main__':
    main()

In [ ]:
llm = ChatOpenAI(
        model = "gpt-3.5-turbo",
        max_tokens=2048,
        temperature=0.5,
)

# prmpt를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template(
    """
    너는 기업 면접관이야.
    {resume} 에 대해서 6가지의 면접 질문을 날카롭게 생성해줘.
    """
)
# prompt 객체와 model 객체릴 파이프(|) 연산자로 연결하고
# invoke 메서드를 이용하여 input을 전달합니다.
# 이를 통해 AI모델이 생성한 메세지를 반환합니다.
chain = prompt | llm
chain.invoke({"resume":vector_db})


In [2]:
!pip install hydralit_components

^C


  Using cached hydralit_components-1.0.10.tar.gz (20.1 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
  Created wheel for hydralit_components: filename=hydralit_components-1.0.10-py3-none-any.whl size=24347449 sha256=52f9327a4e0b7eec4ecdeb6d85e57a2497ff9c5b685438b973410017cc6046ef
  Stored in directory: c:\users\joseu\appdata\local\pip\cache\wheels\5d\5b\e9\3b7a3a5f9f470fd6745417860bc8dc7a68fd6862b19a9290cd
Successfully built hydralit_components


In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import PromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import CommaSeparatedListOutputParser

import os
os.environ['OPENAI_API_KEY'] = "sk-proj-dda7JIkccOeN1l6TtwOtT3BlbkFJWvKSdCKCSzHx4RQgxKKi"

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
splitted_text = text_splitter.split_text(
"""
"최고가 되기 위해서"
금융결제원은 국내 다양한 금융 관련 서비스를 지원하는 최고의 기업입니다.
이런 여러 서비스 운영에 있어 중요한 것은 사용자에게 적절한 UI의 표출, 그리고 효율적인 데이터 처리를 통한 빠른 속도 제공입니다.
저는 이를 배우기 위해 매일 진행 중인 알고리즘 풀이 외에도 기존에 공부했던 JSP에 대해 조금 더 파악하고, spring에 활용하기 전에 배우기 위해 관련 학원에 다니며 공부하였고, 여러 예제를 사용한 Spring 프로젝트를 진행했습니다.
그리고 JPA를 사용하는 여러 방식과 주의점, querydsl 등 많은 부분에의 공부를 해 보았습니다.
백엔드 이외에도 front-end에 활용되는 javascript의 기능 활용을 더 잘 알기 위해, vanilla JS를 사용하여 조금씩 난도를 높여가며 프로젝트를 진행 중입니다.
그렇게 실력을 쌓아 가던 중 운이 좋게도 그동안 제가 공부하였고 앞으로의 진로로 결정하게 된 java spring이 기반이 되는 전산직 사무원을 채용한다는 소식을 들었고, 최고의 기업에서 가장 좋아하는 공부를 할 수 있을 것이라는 기대감을 품고 지원하게 되었습니다.
"""
)
print(f"분할된 청크의수: {len(splitted_text)}")
print(splitted_text)

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
db = Chroma.from_texts(splitted_text, embeddings)

# 질의합니다.
query = "너는 무엇을 했어?"
docs = db.similarity_search(query)


분할된 청크의수: 1
['"최고가 되기 위해서"\n금융결제원은 국내 다양한 금융 관련 서비스를 지원하는 최고의 기업입니다.\n이런 여러 서비스 운영에 있어 중요한 것은 사용자에게 적절한 UI의 표출, 그리고 효율적인 데이터 처리를 통한 빠른 속도 제공입니다.\n저는 이를 배우기 위해 매일 진행 중인 알고리즘 풀이 외에도 기존에 공부했던 JSP에 대해 조금 더 파악하고, spring에 활용하기 전에 배우기 위해 관련 학원에 다니며 공부하였고, 여러 예제를 사용한 Spring 프로젝트를 진행했습니다.\n그리고 JPA를 사용하는 여러 방식과 주의점, querydsl 등 많은 부분에의 공부를 해 보았습니다.\n백엔드 이외에도 front-end에 활용되는 javascript의 기능 활용을 더 잘 알기 위해, vanilla JS를 사용하여 조금씩 난도를 높여가며 프로젝트를 진행 중입니다.\n그렇게 실력을 쌓아 가던 중 운이 좋게도 그동안 제가 공부하였고 앞으로의 진로로 결정하게 된 java spring이 기반이 되는 전산직 사무원을 채용한다는 소식을 들었고, 최고의 기업에서 가장 좋아하는 공부를 할 수 있을 것이라는 기대감을 품고 지원하게 되었습니다.']
"최고가 되기 위해서"
금융결제원은 국내 다양한 금융 관련 서비스를 지원하는 최고의 기업입니다.
이런 여러 서비스 운영에 있어 중요한 것은 사용자에게 적절한 UI의 표출, 그리고 효율적인 데이터 처리를 통한 빠른 속도 제공입니다.
저는 이를 배우기 위해 매일 진행 중인 알고리즘 풀이 외에도 기존에 공부했던 JSP에 대해 조금 더 파악하고, spring에 활용하기 전에 배우기 위해 관련 학원에 다니며 공부하였고, 여러 예제를 사용한 Spring 프로젝트를 진행했습니다.
그리고 JPA를 사용하는 여러 방식과 주의점, querydsl 등 많은 부분에의 공부를 해 보았습니다.
백엔드 이외에도 front-end에 활용되는 javascript의 기능 활용을 더 잘 알기 위해, vanilla JS를 사용하여 조금씩 난도를 높여가며 

In [28]:
def mk_questions(context, resume):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    splitted_text = text_splitter.split_text(resume)

    embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
    db = Chroma.from_texts(splitted_text, embeddings)

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.5, openai_api_key="sk-proj-dda7JIkccOeN1l6TtwOtT3BlbkFJWvKSdCKCSzHx4RQgxKKi")

    output_parser = CommaSeparatedListOutputParser()
    format_instructions = output_parser.get_format_instructions()

    template = '''
    You are an expert AI interviewer.
    Use the given context to generate different predictive interview questions in Korean as many you can.
    Please answer like
    1.....
    2.....
    3.....
    4.....
    Please only generate the questions, don't add any explanations.

    <context>
    {context}
    </context>

    {format_instructions}
    '''

    prompt = PromptTemplate(
        template=template,
        input_variables=["context"],
        partial_variables={"format_instructions": format_instructions},
    )

    qa_chain = create_stuff_documents_chain(llm, prompt)

    retriever = db.as_retriever(
        search_type='similarity',
        search_kwargs={
            'k': 3,  # Select top k search results
        }
    )

    rag_chain = create_retrieval_chain(retriever, qa_chain)
    result = rag_chain.invoke(dict(input=context))
    return result["answer"]

result = mk_questions("머신러닝")
print(result["answer"])

1. 당신이 최고의 기업이 되기 위해 어떤 노력을 해왔습니까?
2. 어떤 기술을 배우기 위해 노력했고, 어떤 프로젝트를 진행해 보았습니까?
3. 자바 스프링을 기반으로 하는 전산직 사무원으로 채용되기 위해 어떤 기대를 갖고 있습니까?
4. 당신이 지원한 최고의 기업에서 어떤 분야에서 더 공부하고 싶은가요?
5. UI의 표출과 데이터 처리 속도를 높이기 위해 어떤 노력을 기울였습니까?


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import PromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import CommaSeparatedListOutputParser


def mk_questions(context, resume):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    splitted_text = text_splitter.split_text(resume)

    embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
    db = Chroma.from_texts(splitted_text, embeddings)

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.5, openai_api_key="sk-proj-dda7JIkccOeN1l6TtwOtT3BlbkFJWvKSdCKCSzHx4RQgxKKi")

    output_parser = CommaSeparatedListOutputParser()
    format_instructions = output_parser.get_format_instructions()

    template = '''
    You are an expert AI interviewer.
    Use the given context to generate different predictive interview questions in Korean as many you can.
    Please answer like
    1.....
    2.....
    3.....
    4.....
    Please only generate the questions, don't add any explanations.

    <context>
    {context}
    </context>

    {format_instructions}
    '''

    prompt = PromptTemplate(
        template=template,
        input_variables=["context"],
        partial_variables={"format_instructions": format_instructions},
    )

    qa_chain = create_stuff_documents_chain(llm, prompt)

    retriever = db.as_retriever(
        search_type='similarity',
        search_kwargs={
            'k': 3,  # Select top k search results
        }
    )

    rag_chain = create_retrieval_chain(retriever, qa_chain)
    result = rag_chain.invoke(dict(input=context))
    return result["answer"]